In [ ]:
from flovopy.asl.wrappers import write_global_heatmap_for_ctag
from pathlib import Path
import pandas as pd
from flovopy.processing.sam import VSAM, DSAM 
from flovopy.asl.config import ASLConfig, tweak_config
# -------------------------- Config --------------------------
# directories
HOME = Path.home()
PROJECTDIR      = HOME / "Dropbox" / "BRIEFCASE" / "SSADenver"
LOCALPROJECTDIR = HOME / "work" / "PROJECTS" / "SSADenver_local"
OUTPUT_DIR      = LOCALPROJECTDIR / "ASL_RESULTS"
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)
INPUT_DIR       = PROJECTDIR / "ASL_inputs" / "biggest_pdc_events"
GLOBAL_CACHE    = PROJECTDIR / "asl_global_cache"
METADATA_DIR    = PROJECTDIR / "metadata" 
STATION_CORRECTIONS_DIR = PROJECTDIR / "station_correction_analysis"

# master files
INVENTORY_XML   = METADATA_DIR / "MV_Seismic_and_GPS_stations.xml"
DEM_DEFAULT     = METADATA_DIR / "MONTSERRAT_DEM_WGS84_MASTER.tif"
GRIDFILE_DEFAULT= METADATA_DIR / "MASTER_GRID_MONTSERRAT.pkl"

# parameters for envelopes and cross-correlation
SMOOTH_SECONDS  = 1.0
MAX_LAG_SECONDS = 8.0
MIN_XCORR       = 0.5

# other parameters
DIST_MODE = "3d" # or 2d. will essentially squash Montserrat topography and stations onto a sea-level plane, ignored elevation data, e.g. for computing distances

# Inventory of Montserrat stations
from obspy import read_inventory
INV     = read_inventory(INVENTORY_XML)
print(f"[INV] Networks: {len(INV)}  Stations: {sum(len(n) for n in INV)}  Channels: {sum(len(sta) for net in INV for sta in net)}")

# Montserrat station corrections estimated from regionals
station_corrections_csv = STATION_CORRECTIONS_DIR / "station_gains_intervals.csv"
annual_station_corrections_csv = STATION_CORRECTIONS_DIR / "station_gains_intervals_by_year.csv"
station_corrections_df = pd.read_csv(station_corrections_csv)
annual_station_corrections_df = pd.read_csv(annual_station_corrections_csv)

# Montserrat pre-defined Grid (from 02 tutorial)
from flovopy.asl.grid import Grid
gridobj = Grid.load(GRIDFILE_DEFAULT)

# Montserrat constants
from flovopy.core.mvo import dome_location, REGION_DEFAULT

topo_kw = {
    "add_labels": True,
    "cmap": "gray",
    "region": REGION_DEFAULT,
    "dem_tif": DEM_DEFAULT,  # basemap shading from your GeoTIFF - but does not actually seem to use this unless topo_color=True and cmap=None
    "frame": True,
    "dome_location": dome_location,
}

DEBUG=False
cfg = ASLConfig(
    inventory=INV,
    output_base=OUTPUT_DIR,
    gridobj=gridobj,
    global_cache=GLOBAL_CACHE,
    station_correction_dataframe=station_corrections_df,
    wave_kind="surface",
    speed=1.5,
    Q=23, 
    peakf=2.0,
    dist_mode="3d", 
    misfit_engine="r2",
    window_seconds=5.0,
    min_stations=5,
    sam_class=VSAM, 
    sam_metric="mean",
    debug=DEBUG,
)
cfg.build()




In [ ]:

res = write_global_heatmap_for_ctag(
    OUTPUT_DIR,
    cfg.tag(),
    node_spacing_m=cfg.gridobj.node_spacing_m,
    topo_kw=topo_kw,
    title=f"Energy Heatmap — {cfg.tag()}",
    include_refined=True,   # set False if you want only the primary
    cmap='turbo',
    scale=3,
    log_scale=True,
    verbose=True,
)

print(res)

In [ ]:
from flovopy.asl.wrappers import (
    gather_post_metrics,
    gather_network_magnitudes,
    build_enhanced_catalogs_for_tag,
)
from flovopy.enhanced.eventrate import EventRateConfig

# Your standard cfg, OUTPUT_DIR, etc. already defined

# 1) Combine CSVs
gather_post_metrics(OUTPUT_DIR, cfg.tag())
gather_network_magnitudes(OUTPUT_DIR, cfg.tag())

# 2) Build catalogs + write EventRate outputs
er_cfg = EventRateConfig(interval="D", rolling=7, ema_alpha=None)
assim = build_enhanced_catalogs_for_tag(
    OUTPUT_DIR,
    cfg.tag(),
    write_event_rates=True,
    er_config=er_cfg,
)

# Access results:
print(f"Primary events: {len(assim['primary'])}, Refined: {len(assim['refined'])}")
print("Primary ER CSV:", (assim["primary_eventrate"] or {}).get("csv"))
print("Refined ER CSV:", (assim["refined_eventrate"] or {}).get("csv"))

In [ ]:
print(assim)

In [ ]:
from flovopy.asl.wrappers import (
    gather_post_metrics,
    gather_network_magnitudes,
    build_enhanced_catalogs_for_tag,
)
from flovopy.enhanced.eventrate import EventRateConfig
LAND_DIR = Path(str(OUTPUT_DIR).replace('ASL_RESULTS', 'ASL_RESULTS_LAND'))
res = write_global_heatmap_for_ctag(
    LAND_DIR,
    cfg.tag(),
    node_spacing_m=cfg.gridobj.node_spacing_m,
    topo_kw=topo_kw,
    title=f"Energy Heatmap — {cfg.tag()}",
    include_refined=True,   # set False if you want only the primary
    cmap='turbo',
    scale=3,
    log_scale=True,
    verbose=True,
)

print(res)

# 1) Combine CSVs
gather_post_metrics(LAND_DIR, cfg.tag())
gather_network_magnitudes(LAND_DIR, cfg.tag())

# 2) Build catalogs + write EventRate outputs
er_cfg = EventRateConfig(interval="D", rolling=7, ema_alpha=None)
assim = build_enhanced_catalogs_for_tag(
    LAND_DIR,
    cfg.tag(),
    write_event_rates=True,
    er_config=er_cfg,
)

# Access results:
print(f"Primary events: {len(assim['primary'])}, Refined: {len(assim['refined'])}")
print("Primary ER CSV:", (assim["primary_eventrate"] or {}).get("csv"))
print("Refined ER CSV:", (assim["refined_eventrate"] or {}).get("csv"))